In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

import utils
import constants

In [2]:
def train_model(actor_model, critic_model, data_dir="data/split/", n_files=25, epochs=1, ckpt_dir="ckpt/aunty"):

    file_order = list(range(1, n_files+1))

    if not os.path.exists(ckpt_dir):
        os.mkdir(ckpt_dir)
    if not os.path.exists(utils.path_join(ckpt_dir, model.name)):
        os.mkdir(utils.path_join(ckpt_dir, model.name))

    checkpoints = [utils.path_join(ckpt_dir, model.name, name) for name in os.listdir(utils.path_join(ckpt_dir, model.name))]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        model = tf.keras.models.load_model(latest_checkpoint)
        n = latest_checkpoint.split("-")[-1][0]
    else:
        n = 0

    processed_files = [filename.split(".")[0] for filename in os.listdir(utils.path_join(data_dir, "processed"))]
    for file in file_order:
        if file <= int(n):
            continue
        filename = f"chess{file}"
        print(f"Training on file {filename}")

        # load data from .csv file
        if filename in processed_files:
            df = pd.read_pickle(utils.path_join(data_dir, "processed", filename + ".pkl"))
        else:
            df = pd.read_csv(utils.path_join(data_dir, filename + ".csv"))

            # translate from fen to obs arr
            print("Processing...", end="")
            df.loc[:, "obs"] = df.loc[:, "board"].map(utils.parse_fen)
            print("complete")

        # convert to numpy arrays
        x = {
            "board": tf.keras.utils.to_categorical(np.array(df["obs_board"].values.tolist()), num_classes=13), 
            "misc":np.array(df["obs_misc"].values.tolist())
        }
        y = {"actor":df["move"].values, "critic":df["outcome"].values}

        # train
        model.fit(x, y, batch_size=64, epochs=epochs, validation_split=0.1)

        # save checkpoint between files
        model.save(utils.path_join(ckpt_dir, model.name, f"{model.name}-{file}.h5"))
    print(f"Training completed. Final save file: {utils.path_join(ckpt_dir, model.name, f'{model.name}-{file}.h5')}")

In [9]:
# critic
activation = "relu"
board_input = tf.keras.Input(shape=(64, 13), name="board")
misc_input = tf.keras.Input(shape=(6), name="misc")

conv_model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((8, 8, 13), input_shape=(64,13)),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.MaxPool2D((2, 2),(1, 1), padding="same"), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256)
], name="conv_critic_model")
conv_output = conv_model(board_input)

dense_inputs = tf.keras.layers.concatenate([conv_output, misc_input])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    tf.keras.layers.Input(262),
    tf.keras.layers.Dense(131, activation=activation),
    tf.keras.layers.Dense(131, activation=activation),
    tf.keras.layers.Dense(1, activation="sigmoid")
], name="dense_critic_model")
dense_output = dense_model(dense_inputs)

critic_model = tf.keras.Model(inputs=[board_input, misc_input], outputs=dense_output, name="critic_model")
critic_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryCrossentropy(),
)

In [10]:
# actor
activation = "relu"
board_input = tf.keras.Input(shape=(64, 13), name="board")
misc_input = tf.keras.Input(shape=(6), name="misc")

conv_model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((8, 8, 13), input_shape=(64,13)),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    tf.keras.layers.MaxPool2D((2, 2),(1, 1), padding="same"), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256)
], name="conv_actor_model")
conv_output = conv_model(board_input)

dense_inputs = tf.keras.layers.concatenate([conv_output, misc_input])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    tf.keras.layers.Input(262),
    tf.keras.layers.Dense(131, activation=activation),
    tf.keras.layers.Dense(131, activation=activation),
    tf.keras.layers.Dense(constants.LEN_UCI_MOVES, activation="softmax")
], name="dense_actor_model")
dense_output = dense_model(dense_inputs)

actor_model = tf.keras.Model(inputs=[board_input, misc_input], outputs=dense_output, name="actor_model")
actor_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = tf.keras.metrics.SparseCategoricalAccuracy(),
)

In [11]:
import chess
actor_model.predict(utils.parse_fen(chess.Board().fen()))

2022-10-24 16:46:02.636337: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(f